In [19]:
from functools import partial
import time

from flax.train_state import TrainState
import jax
import jax.numpy as jnp
import optax

from copy_task import init_ccopy_task, step_supervised_ccopy_task
from models import SupervisedLSTMModel
from training import train_on_sequence

ModuleNotFoundError: No module named 'flax.train_state'

In [15]:
vocab_size = 10
min_seq_len = 1
max_seq_len = 10

tbptt_window = 10

rng = jax.random.PRNGKey(time.time_ns())

In [16]:
model = SupervisedLSTMModel(vocab_size=vocab_size)

init_rnn_state_fn = jax.jit(partial(model.apply, {}, method=model.init_rnn_state))
rnn_state = init_rnn_state_fn()
x = jnp.ones((tbptt_window,), dtype=jnp.int32)

rng, param_gen_key = jax.random.split(rng)
variables = model.init(param_gen_key, x[0], rnn_state)

tx = optax.adam(0.0003)
opt_state = tx.init(variables['params'])

apply_fn = partial(model.forward_seq, model)
tx_update_fn = tx.update

In [17]:
env_state = init_ccopy_task(rng, vocab_size, min_seq_len, max_seq_len)
env_step_fn = step_supervised_ccopy_task

train_step = jax.jit(train_on_sequence, static_argnums=(3, 4, 5, 6))
rng = jax.random.PRNGKey(0)

# key, rng = jax.random.split(rng)
# train_sequence = gen_data(key, var_name_len, var_val_len, n_vars)
loss_hist = []
for i in range(100000):
    key, rng = jax.random.split(rng)
    opt_state, variables, rnn_state, loss = train_step(
        opt_state, variables, train_sequence,
        init_rnn_state_fn, apply_fn, tx_update_fn, tbptt_window,
    )
    loss_hist.append(loss)

    if i % 1000 == 0:
        loss_hist = loss_hist[-1000:]
        print(np.mean(loss_hist))


((Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
  Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0